In [1]:
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set display options for pandas DataFrames to show more columns and widen the display
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Define a list of URLs for tennis data CSV files for the year 2025
links = ['http://tennis-data.co.uk/2025/2025.xlsx']
# Initialize an empty DataFrame to store the combined data
df = pd.DataFrame()
# Loop through each link in the 'links' list
for i, elem in enumerate(links):
    if elem[-4:] == '.zip':
        content = requests.get(elem)
        zf = zipfile.ZipFile(BytesIO(content.content))
        temp = pd.read_excel(zf.open(zf.namelist()[0]))
    else:
        temp = pd.read_excel(elem)
    # Concatenate the temporary DataFrame to the main DataFrame, ignoring the original index
    df = pd.concat([df, temp], ignore_index=True)

# Fill missing values in the 'Best of' column with 3 (assuming default best of 3 sets)
df['Best of'] = df['Best of'].fillna(3)
# Filter out rows where 'WRank' (Winner Rank) is missing
df = df[~df['WRank'].isnull()].reset_index(drop=True)
# Filter out rows where 'LRank' (Loser Rank) is missing
df = df[~df['LRank'].isnull()].reset_index(drop=True)
# Filter out rows where 'W1' (Winner Set 1 games) is missing
df = df[~df['W1'].isnull()].reset_index(drop=True)
# Filter out rows where 'L1' (Loser Set 1 games) is missing
df = df[~df['L1'].isnull()].reset_index(drop=True)
# Fill missing values in set score columns (W2-W5, L2-L5) with 0
df[['W2', 'W3', 'W4', 'W5', 'L2', 'L3', 'L4', 'L5']] = df[['W2', 'W3', 'W4', 'W5', 'L2', 'L3', 'L4', 'L5']].fillna(0)

# Create an 'ind' column to alternate between 0 and 1 for each row
df['ind'] = [(lambda x: x % 2)(x) for x in range(len(df))]

# Define a function to check if a string is a space and return 0 if it is, otherwise return the string
def checkempty(str):
    if str == ' ':
        return 0
    return str

# Apply the checkempty function to 'W3' and 'L3' columns
df['W3'] = df['W3'].apply(checkempty)
df['L3'] = df['L3'].apply(checkempty)
# Convert set score columns to float and then to integer type
df[['W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5']] = df[['W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5']].astype(float).astype(int)

# Create 'Player_1' and 'Player_2' columns based on the 'ind' column (alternating winner and loser)
df['Player_1'] = df.apply(lambda row: row['Winner'] if row['ind'] == 0 else row['Loser'], axis = 1)
df['Player_2'] = df.apply(lambda row: row['Winner'] if row['ind'] == 1 else row['Loser'], axis = 1)
# Create 'Rank_1' and 'Rank_2' columns based on the 'ind' column (alternating winner and loser ranks)
df['Rank_1'] = df.apply(lambda row: row['WRank'] if row['ind'] == 0 else row['LRank'], axis = 1)
df['Rank_2'] = df.apply(lambda row: row['WRank'] if row['ind'] == 1 else row['LRank'], axis = 1)
# Create 'Pts_1' and 'Pts_2' columns based on the 'ind' column (alternating winner and loser points)
df['Pts_1'] = df.apply(lambda row: row['WPts'] if row['ind'] == 0 else row['LPts'], axis = 1)
df['Pts_2'] = df.apply(lambda row: row['WPts'] if row['ind'] == 1 else row['LPts'], axis = 1)
# Create 'Odd_1' and 'Odd_2' columns based on the 'ind' column (alternating winner and loser betting odds)
df['Odd_1'] = df.apply(lambda row: row['B365W'] if row['ind'] == 0 else row['B365L'], axis = 1)
df['Odd_2'] = df.apply(lambda row: row['B365W'] if row['ind'] == 1 else row['B365L'], axis = 1)

# Define a function to format the score string based on the 'ind' column
def score(df):
    if df['ind'] == 0:
        # If 'ind' is 0, format the score as Winner sets - Loser sets for each set
        return str(df['W1']) + '-' + str(df['L1']) + ' ' + str(df['W2']) + '-' + str(df['L2']) + ' ' + str(df['W3']) + '-' + str(df['L3']) +\
 ' ' + str(df['W4']) + '-' + str(df['L4']) + ' ' + str(df['W5']) + '-' + str(df['L5']) + ' '
    # If 'ind' is 1, format the score as Loser sets - Winner sets for each set
    return str(df['L1']) + '-' + str(df['W1']) + ' ' + str(df['L2']) + '-' + str(df['W2']) + ' ' + str(df['L3']) + '-' + str(df['W3']) +\
 ' ' + str(df['L4']) + '-' + str(df['W4']) + ' ' + str(df['L5']) + '-' + str(df['W5']) + ' '


 # Apply the score function to create a 'Score' column and replace '0-0' with an empty string
df['Score'] = df.apply(lambda row: score(row).replace('0-0', ''), axis = 1)

# Create a new DataFrame with selected columns from the original DataFrame
new_df = df[['Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Player_1', 'Player_2','Winner', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2', 'Score', 'Comment']]

# Define a function to replace 'NR' with -1 in a string
def check(str):
    return str.replace('NR', -1)

# Convert the 'Date' column to datetime objects with a specified format
new_df['Date'] = pd.to_datetime(new_df['Date'], format = '%d/%m/%Y')
# Fill any remaining missing values in the DataFrame with -1
new_df = new_df.fillna(-1)
# Apply the 'check' function to replace 'NR' with -1 in specified columns
new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']] = new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']].apply(check)
# Convert specified columns to float and then to integer type
new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']] = new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']].astype(float).astype(int)

# Save the processed DataFrame to a CSV file named 'atp_tennis.csv' without the index
new_df.to_csv('atp_tennis.csv', index=False)

/tmp/ipython-input-3948251974.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date'] = pd.to_datetime(new_df['Date'], format = '%d/%m/%Y')


In [3]:
# Read the saved CSV file back into a DataFrame named 'atp_df'
atp_df = pd.read_csv('/content/atp_tennis.csv')
# Display the first 5 rows of the DataFrame
atp_df.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,Comment
0,Brisbane International,2024-12-29,ATP250,Outdoor,Hard,1st Round,3,Vukic A.,Goffin D.,Vukic A.,68,52,778,1037,2.00,1.80,6-2 6-3,Completed
1,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,O Connell C.,Michelsen A.,Michelsen A.,64,41,795,1245,2.75,1.44,4-6 6-4 6-7,Completed
2,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Bonzi B.,Tabilo A.,Bonzi B.,75,23,730,1943,1.67,2.20,6-7 7-6 6-4,Completed
3,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Rinderknech A.,Nishioka Y.,Nishioka Y.,59,69,927,776,2.50,1.53,3-6 4-6,Completed
4,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Thompson J.,Berrettini M.,Thompson J.,26,34,1745,1380,2.63,1.50,3-6 6-3 6-4,Completed


In [4]:
atp_df.to_csv('/content/drive/MyDrive/Stats/Code/atp_tennis.csv', index=False)

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Stats/Code'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')